# $F_{ST}$ and diversity permutation tests

In [ ]:
#activate
using Pkg
Pkg.activate("../../.")

#import
using NextGenSeqUtils, StatsBase, CSV, DataFrames

#name parsers
particip_func = x->String(split(x,'_')[2])
tissue_func = x->String(split(x, '_')[3])

#distances
hamming(v1::String,v2::String) = sum(collect(v1) .!= collect(v2))

function mean_pairwise_dist(g; dist_func = hamming)
   l=length(g)
   tot = 0.0
   for i in 1:l
       for j in i+1:l
           tot+= dist_func(g[i],g[j])
       end
   end
   return tot/((l*(l-1))/2)
end

function mean_pairwise_dist(g1,g2; dist_func = hamming)
   l1,l2=length(g1),length(g2)
   tot = 0.0
   for i in 1:l1
       for j in 1:l2
           tot+= dist_func(g1[i],g2[j])
       end
   end
   return tot/(l1*l2)
end

#null for permutation tests (stat_func should accept grouped sequences as Array{String,1})
function null_distr(g1, g2, stat_func; reps = 10000)
   l1,l2 = length(g1),length(g2)
   merged = vcat(g1,g2)
   stats = zeros(reps)
   for i in 1:reps
       scrambled = sample(merged,length(merged),replace = false)
       newg1,newg2 = scrambled[1:l1],scrambled[l1+1:end]
       stats[i] = stat_func(newg1,newg2)
   end
   return stats
end

#for FSTtest
function FSTstat_weighted(g1, g2; dist_func = hamming)
   within = (length(g1)*mean_pairwise_dist(g1; dist_func = dist_func) + length(g2)*mean_pairwise_dist(g2; dist_func = dist_func)) / (length(g1) + length(g2))
   between = mean_pairwise_dist(g1,g2; dist_func = dist_func)
   return (between - within) / between
end

function FSTtest_weighted(soqs, group_inds, dist_func; reps = 10000)
   g1, g2 = soqs[group_inds[1]], soqs[group_inds[2]]
   realstat = FSTstat_weighted(g1, g2; dist_func = dist_func)
   return (sum((null_distr(g1, g2, FSTstat_weighted; reps = reps) .>= realstat))/reps, realstat)
end

#for two-tailed diversity test
function div_test_stat(g1, g2; dist_func = hamming)
    first = mean_pairwise_dist(g1; dist_func = dist_func);
    second = mean_pairwise_dist(g2; dist_func = dist_func);
    return abs(second-first)
end

function DivTest2sided(soqs, group_inds, dist_func; reps = 10000)
    g1, g2 = soqs[group_inds[1]], soqs[group_inds[2]]
    realstat = div_test_stat(g1, g2; dist_func = dist_func)
    return (sum((null_distr(g1, g2, div_test_stat; reps = reps) .>= realstat))/reps, realstat)
end

#for grouping
function get_group_inds(arr,func)
    groupings = [func(i) for i in arr]
    groups = union(groupings)
    groupinds = [findall(groupings.==i) for i in groups]
    return groupinds
end

function get_group_pair(arr, func)
    groupings = [func(i) for i in arr]
    groups = union(groupings)
    grouppair = [i => findall(groupings.==i) for i in groups]
    return grouppair
end

function get_comparisons(groups)
    c = []
    for i in 1:length(groups)-1
        for j in i:length(groups)
            if i != j
                push!(c, (i,j))
            end
        end
    end
    return c
end

In [ ]:
#diversity analysis by tissue/participant...
seqnames, seqs = read_fasta_with_names("../../results/sequences.aln");
participant = particip_func.(seqnames);

res = []
@time begin
for p in unique(participant)
    sel_seqs = seqs[participant .== p]
    sel_names = seqnames[participant .== p]
    group_pair = get_group_pair(sel_names, tissue_func);
    
    all_group_inds = getindex.(group_pair, 2)
    all_group_ids = getindex.(group_pair, 1)
    all_group_sizes = length.(all_group_inds)
    
    if length(all_group_inds) == 1
        #skip if only one tissue... 
        push!(res, (p, all_group_ids[1], all_group_sizes[1],missing, missing, missing, missing))
    else
        #compare across tissues... 
        comparisons = get_comparisons(all_group_inds)
        for c in comparisons
            group_sizes = (all_group_sizes[c[1]], all_group_sizes[c[2]])
            group_ids = (all_group_ids[c[1]], all_group_ids[c[2]])
            group_inds = (all_group_inds[c[1]], all_group_inds[c[2]])
            if sum(group_sizes .== 1) > 0
                #skip if only one sequence for selected tissues...
                push!(res, (p, all_group_ids[c[1]], group_sizes[1], all_group_ids[c[2]], group_sizes[2], missing, missing))
            else
                #run test... 
                pval,stat = DivTest2sided(sel_seqs, group_inds, hamming; reps = 10000)
                push!(res, (p, all_group_ids[c[1]], group_sizes[1], all_group_ids[c[2]], group_sizes[2], pval, stat))
            end
        end
    end
end
end

div_results = DataFrame(res)
rename!(div_results, [:participant, :tissue_1, :n_tissue_1, :tissue_2, :n_tissue_2, :pval, :stat])

In [ ]:
CSV.write("diversity.tsv", div_results; delim = '\t')

In [ ]:
#FST analysis by tissue/participant...
seqnames, seqs = read_fasta_with_names("../../results/sequences.aln");
participant = particip_func.(seqnames);

res = []
@time begin
for p in unique(participant)
    sel_seqs = seqs[participant .== p]
    sel_names = seqnames[participant .== p]
    group_pair = get_group_pair(sel_names, tissue_func);
    
    all_group_inds = getindex.(group_pair, 2)
    all_group_ids = getindex.(group_pair, 1)
    all_group_sizes = length.(all_group_inds)
    
    if length(all_group_inds) == 1
        push!(res, (p, all_group_ids[1], all_group_sizes[1],missing, missing, missing, missing))
    else
        #compare across tissues... 
        comparisons = get_comparisons(all_group_inds)
        for c in comparisons
            group_sizes = (all_group_sizes[c[1]], all_group_sizes[c[2]])
            group_ids = (all_group_ids[c[1]], all_group_ids[c[2]])
            group_inds = (all_group_inds[c[1]], all_group_inds[c[2]])
                
            if sum(group_sizes .== 1) > 0
                #skip if only one sequence for selected tissues...
                push!(res, (p, all_group_ids[c[1]], group_sizes[1], all_group_ids[c[2]], group_sizes[2], missing, missing))
            else
                #run test...
                pval,stat = FSTtest_weighted(sel_seqs,group_inds,hamming; reps = 10000)
                push!(res, (p, all_group_ids[c[1]], group_sizes[1], all_group_ids[c[2]], group_sizes[2], pval, stat))
            end
        end
    end
end
end

fst_results = DataFrame(res)
rename!(fst_results, [:participant, :tissue_1, :n_tissue_1, :tissue_2, :n_tissue_2, :pval, :stat])

In [ ]:
CSV.write("FST.tsv", fst_results; delim = '\t')